# *Furry* Dreambooth
### Notebook implementation by Joe Penna (@MysteryGuitarM on Twitter) - Improvements by David Bielejeski
### Furry specific modifications by dogarrowtype#7324
### Join us at: https://discord.gg/furrydiffusion

Dreambooth is a technique that runs on top of StableDiffusion. It teaches the model the details of your specific character, and if done right, can reproduce them with high accuracy.


## This notebook requires a GPU with 24GB of VRAM. A 3090 or an A5000 on runpod will do the job.
Pick a runpod with a fast upload speed, to download the model at the end faster. Choose the standard pytorch template.

### Instructions
- Run each code cell, follow the instructions, but choose whether you want to use regularization images.

Modified from:
https://github.com/JoePenna/Dreambooth-Stable-Diffusion

This notebook has been designed and tested to work on https://runpod.io

Be careful, the predefined model is known to produce nsfw content. DO NOT use if you are below the legal age of majority in your country.

## Build Environment

In [ ]:
!git clone https://github.com/JoePenna/Dreambooth-Stable-Diffusion
%cd /workspace/Dreambooth-Stable-Diffusion
!git checkout 605faeaa2f0656bf31ab79749fb5d96b3ca2e69f

In [ ]:
# BUILD ENV
!pip install omegaconf
!pip install einops
!pip install pytorch-lightning==1.6.5
!pip install test-tube
!pip install transformers
!pip install kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install setuptools==59.5.0
!pip install pillow==9.0.1
!pip install torchmetrics==0.6.0
!pip install -e .
!pip install protobuf==3.20.1
#!pip install gdown
!pip install -qq diffusers["training"]==0.3.0 transformers ftfy
!pip install -qq "ipywidgets>=7,<8"
!pip install huggingface_hub
!pip install ipywidgets==7.7.1
!pip install captionizer==1.0.1

## Download the model to train on top of
Swap this link out here, if you want to use something other than ye18.

In [ ]:
!wget https://sexy.canine.wf/file/yiffy-ckpt/yiffy-e18.ckpt -O /workspace/Dreambooth-Stable-Diffusion/model.ckpt

# Regularization Images (No longer recommended)
## Skip this part unless you know what it does.
### Expand if you really want to.

This is NOT the part that contains the character you want to train on. It is a separate step that helps guide the model toward what you want, but has little to do with your desired character.

For furry content, ~~regularization images help. They (seem) to prime the model to focus on the area you'd like to change.~~

## Regularization images seem to cause more harm than help for furry content. I no longer advise using them.

### But you can still use them if you want. Just expand this block.

Change "self_generated_files_prompt" to be closest to what you want. 

**For instance: change species, gender, color.**

Try to keep it close to the example and simple. Don't go heavy on the detail.

The prompt system here is not as advanced as Automatic1111, so don't do anything fancy.

In [ ]:
# GENERATE 200 images - Optional
uses_reg = True
self_generated_files_prompt = "e621 safe solo anthro male canine grey fur"
self_generated_files_count = 200

!python scripts/stable_txt2img.py \
 --seed 10 \
 --ddim_eta 0.0 \
 --n_samples 1 \
 --n_iter {self_generated_files_count} \
 --scale 10.0 \
 --ddim_steps 50 \
 --ckpt model.ckpt \
 --prompt "{self_generated_files_prompt}"


In [ ]:
dataset="dataset1"
!mkdir -p regularization_images/{dataset}
!mv /workspace/Dreambooth-Stable-Diffusion/outputs/txt2img-samples/samples/*.* regularization_images/{dataset}

In [ ]:
# Zip up the files for downloading and reuse.
# Download this file locally so you can reuse during another training on this dataset
!apt update
!apt install -y zip
!zip -r regularization_images.zip regularization_images/{dataset}

## OR Upload regularization images generated separately to
`/workspace/Dreambooth-Stable-Diffusion/regularization_images/dataset1`

Don't do both.

In [ ]:
uses_reg = True
dataset="dataset1"
!mkdir -p regularization_images/{dataset}

Generate them on your own pc, gather them, and upload to the folder. If you *really* want to skip regularization, delete `--reg_data_root "{reg_data_root}"` from the training block.

# Training data
# Upload your training images

In [ ]:
!mkdir -p /workspace/Dreambooth-Stable-Diffusion/training_images

Upload 100-200 images of your character to

```
/workspace/Dreambooth-Stable-Diffusion/training_images
```

Be sure to upload an *even* amount of images, otherwise the training inexplicably stops at 1500 steps.

## The images should be:

- Solos only!
- Square.
- 512x512.
- Cropped to contain the most recognizable part of your character.
- Don't worry about legs/arms being cropped off, the most important part is the head. The model will make up the rest.
- A variety of angles and shots. The model can become overfitted if you only do frontal shots, for instance.
- If your character has very distinctive features or markings you want to preserve, make sure they show up in the data often.
- Variations of the same image (alt versions/edits) are fine, but don't do too many of the same.
- DON'T do black and white or uncolored sketches.
- Get as close as possible to the output you'd like to see.

A good idea is to use imagemagick for automated cropping. No, it's not the fancy kind that uses algorithms to focus on faces, but it totally works.

You're mostly on your own for cropping, but here's some...
## Hints for cropping commands:
Windows:

`magick *.* -gravity North -crop 1:1 -resize 512x512 -format png ..\raw1\image-%d.png`

Linux:

`find -maxdepth 1 -type f -execdir magick '{}' -gravity North -crop 1:1 -resize 512x512 ../ready1/'{}.png' \;`

https://imagemagick.org/script/download.php


## Use BIRME, it's by far the best option for bulk crop + resize:
https://www.birme.net/?target_width=512&target_height=512&rename=ORIGINAL-NAME_&border_width=-1

# Training

## This is where the magic happens.

Some key things to change:
- `"project_name"`: Change project name to whatever you want. Avoid spaces for everything.
- `"max_training_steps"`: 2000-4000 is a good range. Choose around 2000 if you have 100 images, and around 4000 if you have 200 images.
- `"class_word"`: Keep as anthro. Or feral. Or... whatever kind of thing you're doing. But it should be extremely simple, and something the model already recognizes easily. Only matters if using regularization images.
- `"token"`: Pick a longer, made-up word the model has never seen before. Can really be anything, but make it something like "blaiddwolfyfromeldenring". "Charname+universe" is easier to remember, imo. This is what you will type in to activate it, so... don't make it insane.

Once you've got those set, you're ready to train. It should take about an hour on a 3090 for 2500 steps.

Do not worry. It will stop at exactly the number of steps you set, even if the progress bar there looks otherwise. The estimate is usually wildly wrong. There is a separate internal counter that is not shown.

In [ ]:
# Training

# This is just the name for the training run, not the token.
project_name = "blaidd_1"

# MAX STEPS
# How many steps do you want to train for?
max_training_steps = 2500

# Match class_word to the category of the regularization images you chose above.
class_word = "anthro" # Anthro or feral are the best choices.

# This is the unique token you are teaching the stable diffusion model.
token = "blaiddwolfyfromeldenring" #Can be anything, but pick something rare and easy to remember/type.


dataset="dataset1"
img_data_root = "/workspace/Dreambooth-Stable-Diffusion/training_images/"
reg_data_root = "/workspace/Dreambooth-Stable-Diffusion/regularization_images/" + dataset

!rm -rf training_images/.ipynb_checkpoints

if uses_reg:
    !python "main.py" \
    --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
    -t \
    --actual_resume "model.ckpt" \
    --reg_data_root "{reg_data_root}" \
    -n "{project_name}" \
    --gpus 0, \
    --data_root "{img_data_root}" \
    --max_training_steps {max_training_steps} \
    --class_word "{class_word}" \
    --token "{token}" \
    --no-test
else:
    !python "main.py" \
    --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
    -t \
    --actual_resume "model.ckpt" \
    -n "{project_name}" \
    --gpus 0, \
    --data_root "{img_data_root}" \
    --max_training_steps {max_training_steps} \
    --token "{token}" \
    --no-test

## Copy and name the checkpoint file

Also prints the sha256sum, very helpful for verification/sharing your dreambooth model publicly. Take note of it.

In [ ]:
# Copy the checkpoint into the `trained_models` folder

directory_paths = !ls -d logs/*
last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"
training_images = !find training_images/*
date_string = !date +"%Y-%m-%dT%H-%M-%S"
file_name = date_string[-1] + "_" + project_name + "_" + str(len(training_images)) + "_training_images_" +  str(max_training_steps) + "_max_training_steps_" + token + "_token_" + class_word + "_class_word.ckpt"

file_name = file_name.replace(" ", "_")

!mkdir -p trained_models
!mv "{last_checkpoint_file}" "trained_models/{file_name}"
!sha256sum "trained_models/{file_name}"

print("Download your trained model file from trained_models/" + file_name + " and use in your favorite Stable Diffusion repo!")

## Upload checkpoint directly to Pixeldrain
Saves you time and money if you have a slow download speed. Slightly risky, but for some, very worth it. If your rented pod has a fast internet connection, it will upload to pixeldrain very quickly.

### It won't show any output while it is uploading. But if it worked, it will print out a link at at the bottom that you can use to download/share your model.

Take extra care, this method can go wrong in any number of ways.

In [ ]:
!wget https://github.com/ManuelReschke/go-pd/releases/download/v1.2.1/go-pd_1.2.1_linux_amd64.tar.gz -O ./go-pd.tar.gz
!tar -xf ./go-pd.tar.gz go-pd
!./go-pd upload "/workspace/Dreambooth-Stable-Diffusion/trained_models/{file_name}"